In [4]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import time

# --- Configuration ---
# You can change the ticker symbol to any stock you want to analyze
ticker_symbol = "NVDA"

# --- Date Calculation ---
# Automatically calculate the start and end dates for the last 30 days
end_date = datetime.now()
start_date = end_date - timedelta(days=30)

# Format dates into the 'YYYY-MM-DD' string format required by yfinance
start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')

print(f"--- Preparing to download 1-minute data for {ticker_symbol} ---")
print(f"Date Range: {start_date_str} to {end_date_str}\n")


# --- Data Download Loop ---
# List to hold the data chunks from each API call
all_data = []

# Loop through the date range in 7-day chunks (to respect API limits)
current_start = start_date
while current_start < end_date:
    # Define the end of the current chunk (7 days later)
    current_end = current_start + timedelta(days=7)

    # Ensure the last chunk doesn't go past the overall end date
    if current_end > end_date:
        current_end = end_date

    print(f"Fetching data from {current_start.strftime('%Y-%m-%d %H:%M')} to {current_end.strftime('%Y-%m-%d %H:%M')}...")

    try:
        # Fetch data for the small chunk
        stock_data = yf.download(
            tickers=ticker_symbol,
            start=current_start,
            end=current_end,
            interval="1m",
            auto_adjust=True, # Set to True to silence the warning and get adjusted prices
            progress=False
        )

        # If data was successfully downloaded, add it to our list
        if not stock_data.empty:
            all_data.append(stock_data)
        else:
            print("No data found for this chunk.")

    except Exception as e:
        print(f"An error occurred: {e}")

    # Be polite to the API and avoid getting rate-limited
    time.sleep(1)

    # Move to the start of the next chunk
    current_start = current_end

# --- Final Processing ---
# Concatenate all the downloaded dataframes and save to a CSV file
if all_data:
    # Combine all the chunks into a single DataFrame
    final_df = pd.concat(all_data)

    # --- Save the data to a CSV file ---
    file_name = f"{ticker_symbol}_1m_data_last_30_days.csv"
    final_df.to_csv(file_name)

    print(f"\n✅ Successfully downloaded and saved data to '{file_name}'")
    print("\n--- First 5 Rows ---")
    print(final_df.head())
    print("\n--- Last 5 Rows ---")
    print(final_df.tail())
else:
    print("\nNo data was downloaded. This could be due to API issues or no trading data in the last 30 days (e.g., weekends/holidays).")



--- Preparing to download 1-minute data for NVDA ---
Date Range: 2025-06-17 to 2025-07-17

Fetching data from 2025-06-17 21:40 to 2025-06-24 21:40...


KeyboardInterrupt: 